# RoBERTa Fine Tunning For Helpfulness Prediction

# Load Datasets

## Import Needed Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Read CSV Files

In [2]:
train_df = pd.read_csv("train.csv")
train_df

,Unnamed: 0,_id,text,label,helpful,unhelpful
0,0,657855651a6d2c7052a63c2e,The selection here is okay if you're making ga...,helpful,1,0
1,1,657859921a6d2c7052a64d48,"Now THIS is a restaurant. Small, tidy, excell...",unhelpful,0,1
2,2,657855651a6d2c7052a63de3,In the last five minutes I had two men knock o...,helpful,1,0
3,3,657859921a6d2c7052a6529f,I love Potbelly's. I've eaten there several t...,unhelpful,0,1
4,4,657855641a6d2c7052a63332,"This is the best thai cuisine around, the curr...",helpful,1,0
...,...,...,...,...,...,...
7995,7995,657859921a6d2c7052a64d6e,Service was good and friendly. Food was good ...,unhelpful,0,1
7996,7996,657855651a6d2c7052a6440c,This place is a complete joke. I bought 4 tire...,helpful,1,0
7997,7997,657855651a6d2c7052a64353,I waited a week to post my review of White Dog...,helpful,1,0
7998,7998,657855641a6d2c7052a633cd,"We had a Groupon for Euphoria, so my husband a...",helpful,1,0


In [3]:
train_df["label"].value_counts()

label
helpful      4000
unhelpful    4000
Name: count, dtype: int64

In [4]:
test_df = pd.read_csv("test.csv")
test_df

,Unnamed: 0,_id,text,label,helpful,unhelpful
0,0,657859921a6d2c7052a64801,I was out in St. Louis on a business trip and ...,unhelpful,0,1
1,1,657859921a6d2c7052a647aa,Worst omelet I have ever eaten. With the upcha...,unhelpful,0,1
2,2,657855651a6d2c7052a63749,I love this place! It is my favorite go to pla...,helpful,1,0
3,3,657855651a6d2c7052a6408a,Came to Tamarind for lunch on Sunday afternoon...,helpful,1,0
4,4,657859921a6d2c7052a6554d,"Easter Sunday, over crowded at 6:00 PM, staff ...",unhelpful,0,1
...,...,...,...,...,...,...
1995,1995,657859921a6d2c7052a65259,Love this dealership. Bought my CRV there a fe...,unhelpful,0,1
1996,1996,657855651a6d2c7052a63ce4,We stopped at Tako due to the positive reviews...,helpful,1,0
1997,1997,657855651a6d2c7052a63d89,I had a quick get together with a couple of fr...,helpful,1,0
1998,1998,657859921a6d2c7052a64b23,"After a long week, a girl friend and I decided...",unhelpful,0,1


In [5]:
test_df["label"].value_counts()

label
unhelpful    1000
helpful      1000
Name: count, dtype: int64

## Build PyTorch Dataset Class

In [6]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import torch

In [7]:
class Helpfulness_Dataset(Dataset):
    
    def __init__(self, data, tokenizer, attributes, max_token_len):
        self.data = data
        self.tokenizer = tokenizer
        self.attributes = attributes
        self.max_token_len = max_token_len
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        item = self.data.iloc[index]
        text = item.text
        attributes = torch.FloatTensor(item[self.attributes])
        tokens = self.tokenizer.encode_plus(text,
                                            add_special_tokens=True,
                                            return_tensors="pt",
                                            truncation=True,
                                            max_length=self.max_token_len,
                                            padding="max_length",
                                            return_attention_mask=True)
        
        return {
            "input_ids": tokens.input_ids.flatten(),
            "attention_mask": tokens.attention_mask.flatten(),
            "labels": attributes
        }

## Create Train And Test PyTorch Datasets

In [8]:
attributes = ["helpful", "unhelpful"]
model_name = "roberta-base"
max_token_length = 512
tokenizer = AutoTokenizer.from_pretrained(model_name)
hfs_ds_train = Helpfulness_Dataset(train_df, tokenizer, attributes, max_token_length)
hfs_ds_val = Helpfulness_Dataset(test_df, tokenizer, attributes, max_token_length)

## Tokization Example

### Helpful Review

In [9]:
hfs_ds_train.data.iloc[0].text

"The selection here is okay if you're making garments, quilting (even rag) is a no go...ever! Fleece selection and price is great and one of the reasons to go here. Warm and natural batting is more expensive than Hancock or anywhere else except high end quilt shops. Price is okay on fabric for garments and there's a nice selection of silk/rayon/syn but the quality of some fabrics reflects the low everyday price. My opinion is that these are 2nd and 3rd cut fabrics not top quality. If you use a coupon or go for a sale at Hancock you can get better quality fabric at close to the same price. Plus their notions are about non-existant, just try to match thread to fabric there...good luck! \n\nThe real reason for a low rating is the customer service! I agree with the woman from San Jose who said it was bad unless they know you. That was my experience also. I took my daughter here and after she left me to go to the bathroom she came back freaked out. She said one of the ladies who worked ther

In [10]:
hfs_ds_train.__getitem__(0)

{'input_ids': tensor([    0,   133,  4230,   259,    16,  8578,   114,    47,   214,   442,
         30625,     6,  2677,   718,  2577,    36, 12963, 31179,    43,    16,
            10,   117,   213,   734,  6294,   328,   274,  7445,  1755,  4230,
             8,   425,    16,   372,     8,    65,     9,     5,  2188,     7,
           213,   259,     4, 19516,     8,  1632,  8032,    16,    55,  3214,
            87, 19632,    50,  4558,  1493,  4682,   239,   253,  2677, 10325,
          6464,     4,  3655,    16,  8578,    15, 10199,    13, 30625,     8,
            89,    18,    10,  2579,  4230,     9, 22288,    73,  5022,   261,
            73, 38972,    53,     5,  1318,     9,   103, 26348,  6771,     5,
           614,  7476,   425,     4,  1308,  2979,    16,    14,   209,    32,
           132,  1187,     8,   155,  2586,   847, 26348,    45,   299,  1318,
             4,   318,    47,   304,    10, 22939,    50,   213,    13,    10,
          1392,    23, 19632,    47,   

### Unhelpful Review

In [11]:
hfs_ds_val.data.iloc[0].text

"I was out in St. Louis on a business trip and was looking forward to trying Pappy's (due to all the hype from yelp and the travel channel).  I ordered a full rack of ribs with deep fried corn and green beans.  The deep fried corn was awesome but I was pretty dissapointed in the ribs.  I'm not sure if it was because I came later in the day around 6 pm (shouldn't matter) but my ribs were pretty dry.  I've had my fair share of dry rub ribs and expect them to still be tender and moist.  My travel partners were also not too impressed with the ribs from Pappy's.  Maybe it was an off day for them, but this is definitely not close to the best ribs that I've had."

In [12]:
hfs_ds_val.__getitem__(0)

{'input_ids': tensor([    0,   100,    21,    66,    11,   312,     4,  3217,    15,    10,
           265,  1805,     8,    21,   546,   556,     7,   667,   221, 31953,
            18,    36, 17193,     7,    70,     5, 14761,    31,  1423,   523,
           642,     8,     5,  1504,  4238,   322,  1437,    38,  2740,    10,
           455, 20004,     9, 21443,    19,  1844, 16708,  7636,     8,  2272,
         13095,     4,  1437,    20,  1844, 16708,  7636,    21,  6344,    53,
            38,    21,  1256, 14863,  1115, 26427,    11,     5, 21443,     4,
          1437,    38,   437,    45,   686,   114,    24,    21,   142,    38,
           376,   423,    11,     5,   183,   198,   231,  4751,    36, 17276,
           282,    75,   948,    43,    53,   127, 21443,    58,  1256,  3841,
             4,  1437,    38,   348,    56,   127,  2105,   458,     9,  3841,
         14204, 21443,     8,  1057,   106,     7,   202,    28,  8780,     8,
         34257,     4,  1437,  1308,  1

# 2. Data Module

## Import Needed Libraries

In [13]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

## Creating PyTorch Data Module

In [14]:
class Helpfulness_Data_Model(pl.LightningDataModule):
    
    def __init__(self, attributes, batch_size, max_token_length, model_name):
        super().__init__()
        self.attributes = attributes
        self.batch_size = batch_size
        self.max_token_length = max_token_length
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
    def setup(self, stage = None):
        if stage in (None, "fit"):
            self.train_dataset = Helpfulness_Dataset(train_df, tokenizer, attributes, 512)
            self.val_dataset = Helpfulness_Dataset(test_df, tokenizer, attributes, 512)
        if stage == "predict":
            self.val_dataset = Helpfulness_Dataset(test_df, tokenizer, attributes, 512)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          num_workers=4,
                          shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          num_workers=4,
                          shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          num_workers=4,
                          shuffle=False)

## Create Instance Of Our Data Module And Set It Up

In [15]:
attributes = ["helpful", "unhelpful"]
model_name = "roberta-base"
batch_size = 8
max_token_length = 512
hfs_data_module = Helpfulness_Data_Model(attributes, batch_size, max_token_length, model_name)
hfs_data_module.setup()
dl = hfs_data_module.train_dataloader()

## Number Of Batches

In [16]:
len(dl)

1000

# 3. Model

## Import Needed Libraries

In [17]:
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
import torch.nn as nn
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F

2023-12-23 02:11:07.468049: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 02:11:07.496683: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-23 02:11:08.106008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Helpfulness Classifier Class

In [18]:
class Helpfulness_Classifier(pl.LightningModule):
    
    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.pretrained_model = AutoModel.from_pretrained(config["model_name"], return_dict=True)
        self.hidden= nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        self.classification = nn.Linear(self.pretrained_model.config.hidden_size, self.config["n_labels"])
        torch.nn.init.xavier_uniform_(self.hidden.weight)
        torch.nn.init.xavier_uniform_(self.classification.weight)
        self.loss_fun = nn.BCEWithLogitsLoss(reduction="mean")
        self.dropout = nn.Dropout()
        
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.pretrained_model(input_ids = input_ids, attention_mask = attention_mask)
        pooled_output = torch.mean(output.last_hidden_state, 1)
        pooled_output = self.hidden(pooled_output)
        pooled_output = self.dropout(pooled_output)
        pooled_output = F.relu(pooled_output)
        logits = self.classification(pooled_output)
        loss = 0 
        if labels is not None:
            loss = self.loss_fun(logits.view(-1, self.config["n_labels"]), labels.view(-1, self.config["n_labels"]))
        return loss, logits
    
    def training_step(self, batch, batch_index):
        loss, logits = self(**batch)
        self.log("train loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": logits, "labels": batch["labels"]}
    
    def validations_step(self, batch, batch_index):
        loss, logits = self(**batch)
        self.log("validation loss", loss, prog_bar=True, logger=True)
        return {"val_loss": loss, "predictions": logits, "labels": batch["labels"]}
    
    def predict_step(self, batch, batch_index):
        _, logits = self(**batch)
        return logits
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config["lr"], weight_decay=self.config["w_decay"])
        total_steps = self.config["train_size"] / self.config["bs"]
        warmup_steps = math.floor(total_steps * self.config["warmup"])
        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        return [optimizer], [scheduler]
    

In [19]:
config = {
    "model_name": "roberta-base",
    "n_labels": len(attributes),
    "bs": 8,
    "lr": 2e-5,
    "warmup": 0.2,
    "w_decay": 0.001,
    "train_size": len(hfs_data_module.train_dataloader()),
    "n_epochs": 4
}

model = Helpfulness_Classifier(config)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
idx = 0

input_ids = hfs_ds_train.__getitem__(idx)["input_ids"]
am = hfs_ds_train.__getitem__(idx)["attention_mask"]
labels = hfs_ds_train.__getitem__(idx)["labels"]

loss, output = model(input_ids.unsqueeze(dim=0), am.unsqueeze(dim=0), labels.unsqueeze(dim=0))

In [21]:
loss, output

(tensor(0.6558, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor([[-0.4042, -0.7219]], grad_fn=<AddmmBackward0>))

## Train

In [22]:
torch.set_float32_matmul_precision('medium')
hfs_data_module = Helpfulness_Data_Model(attributes, config["bs"], max_token_length, model_name)
hfs_data_module.setup()

model = Helpfulness_Classifier(config)

trainer = pl.Trainer(max_epochs=config["n_epochs"], num_sanity_val_steps=50)
trainer.fit(model, hfs_data_module)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type              | Params
-------------------------------------------------------
0 | pretrained_model | RobertaModel      | 124 M 
1 | hidden           | Linear            | 590 K 
2 | classification   | Linear            | 1.5 K 
3 | loss_fun         | BCEWithLogitsLoss | 0     
4 | dropout          | Dropout           | 0     
-------------------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.951   Total estim

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Training: |                                                              | 0/? [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

`Trainer.fit` stopped: `max_epochs=4` reached.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Predict / Eval

### Import Neded Libraries

In [23]:
def classify_reviews(model, dm):
    preictions = trainer.predict(model, datamodule=dm)
    flattened_prediction = np.stack([torch.sigmoid(torch.Tensor(p)) for batch in preictions for p in batch])
    return flattened_prediction

In [24]:
hfs_data_module.val_dataset.data[["helpful", "helpful"]]

,helpful,helpful
0,0,0
1,0,0
2,1,1
3,1,1
4,0,0
...,...,...
1995,0,0
1996,1,1
1997,1,1
1998,0,0


In [25]:
predictions = classify_reviews(model, hfs_data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Predicting: |                                                            | 0/? [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
predictions_df = pd.DataFrame(predictions)
predictions_df["label"] = "unhelpful"
predictions_df.loc[predictions_df[0] > 0.5, 'label'] = 'helpful'
predictions_df

,0,1,label
0,0.286865,0.735822,unhelpful
1,0.133672,0.852289,unhelpful
2,0.125041,0.895952,unhelpful
3,0.397454,0.658083,unhelpful
4,0.222300,0.777611,unhelpful
...,...,...,...
1995,0.071292,0.898162,unhelpful
1996,0.277308,0.727969,unhelpful
1997,0.761461,0.288247,helpful
1998,0.279996,0.680565,unhelpful


In [27]:
predictions_df["label"].value_counts()

label
unhelpful    1157
helpful       843
Name: count, dtype: int64

In [28]:
def generate_confusion_matrix(predictions, ground_truth_labels):
    
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for i in range(predictions.shape[0]):
        if ground_truth_labels.iloc[i]["label"] == "helpful":
            if predictions.iloc[i]["label"] == "helpful":
                tp += 1
            else:
                fp +=1
        else:
            if predictions.iloc[i]["label"] == "unhelpful":
                tn += 1
            else:
                fn +=1
            
    return tp, fp, tn, fn

In [29]:
tp, fp, tn, fn = generate_confusion_matrix(predictions_df,  hfs_data_module.val_dataset.data)

print(f"True Positive = {tp}")
print(f"False Positive = {fp}")
print(f"True negative = {tn}")
print(f"False negative = {fn}")

with open("Results/RoBERTa-Base-Met.txt", "w") as f:
    f.write(f"True Positive = {tp}\n")
    f.write(f"False Positive = {fp}\n")
    f.write(f"True negative = {tn}\n")
    f.write(f"False negative = {fn}\n")
    f.write(f"Total = {tp + fp + tn + fn }")

True Positive = 694
False Positive = 306
True negative = 851
False negative = 149


In [30]:
def gen_results(tp, fp, tn, fn):
    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) != 0 else 0.0
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0.0
    
    return accuracy, precision, recall, f1_score, specificity

In [31]:
accuracy, precision, recall, f1_score, specificity = gen_results(tp, fp, tn, fn)

print(f"Accuracy = {accuracy}")
print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"F1 Score = {f1_score}")
print(f"Specificity = {specificity}")

with open("Results/RoBERTa-Base-Results.txt", "w") as f:
    f.write(f"Accuracy = {accuracy}\n")
    f.write(f"Precision = {precision}\n")
    f.write(f"Recall = {recall}\n")
    f.write(f"F1 Score = {f1_score}\n")
    f.write(f"Specificity = {specificity}")

Accuracy = 0.7725
Precision = 0.694
Recall = 0.8232502965599051
F1 Score = 0.7531199131850244
Specificity = 0.7355229040622299
